In [1]:
import dill as pickle
import aesara.tensor as aet
import pandas as pd
import pycmtensor as cmt
from pycmtensor.pycmtensor import PyCMTensorModel, Beta, Weights
from pycmtensor.functions import logit, neg_loglikelihood
from pycmtensor.optimizers import Adam
from pycmtensor.results import Results

swissmetro = pd.read_csv("data/swissmetro.dat", sep="\t")
db = cmt.Database("swissmetro", swissmetro, choiceVar="CHOICE")

In [2]:
# Removing some observations
exclude = ((db.variables["PURPOSE"] != 1) * (db.variables["PURPOSE"] != 3) + (db.variables["CHOICE"] == 0)) > 0
db.remove(exclude)

# additional steps to format database
db.data["CHOICE"] -= 1 # set the first choice to 0
db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
    'SM_CO', 'SM_TT'], default=100., verbose=False)


class ResLogitLayer:
    def __init__(self, input, w_in, w_out, activation_in=None, activation_out=None):
        assert w_in.shape.eval()[1] == w_out.shape.eval()[0]
        if isinstance(input, (list, tuple)):
            assert len(input) == w_in.shape.eval()[0], f"index.0 of w_in must be of the same length as input"
            input = aet.concatenate(input, axis=1)

        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        self.w_in = w_in()
        self.w_out = w_out()
            
        h = aet.sigmoid(aet.dot(input, self.w_in))
        output = aet.sigmoid(aet.dot(h, self.w_out))
        self.input = input
        self.weights = [self.w_in, self.w_out]
        self.output = output + input

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"


class MNLmodel(PyCMTensorModel):
    def __init__(self, db):
        super().__init__()
        self.name = "myModel"
        self.inputs = db.inputs()  # keep track of inputs

        # update global variables from database
        for var in self.inputs:
            globals().update({var.name: var})

        # declare model params here
        b_cost = Beta("b_cost", 0.0, None, None, 0)
        b_time = Beta("b_time", 0.0, None, None, 0)
        asc_train = Beta("asc_train", 0.0, None, None, 0)
        asc_car = Beta("asc_car", 0.0, None, None, 0)
        asc_sm = Beta("asc_sm", 0.0, None, None, 1)

        W1 = Weights("ResNet_01a", (3, 10), 0, True)
        W2 = Weights("ResNet_01b", (10, 3), 0, True)

        # append model params to self.params list
        self.append_to_params(locals())
        # self.append_to_params([b_cost, b_time, asc_train, asc_car, asc_sm, W1, W2])

        # Definition of the utility functions
        U_1 = b_cost * TRAIN_CO + b_time * TRAIN_TT + asc_train
        U_2 = b_cost * SM_CO + b_time * SM_TT + asc_sm
        U_3 = b_cost * CAR_CO + b_time * CAR_TT + asc_car
        U = [U_1, U_2, U_3]
        rh = ResLogitLayer(U, W1, W2)

        # definition of the choice output
        self.y = CHOICE

        # symbolic expression for the choice model
        self.p_y_given_x = logit(rh.output, [TRAIN_AV, SM_AV, CAR_AV])

        # declare Regularizers here:
        # L1 regularization cost
        self.L1 = abs(b_cost()) + abs(b_time())

        # L2 regularization cost
        self.L2 = b_cost() ** 2 + b_time() ** 2

        # symbolic expression for the cost fuction
        self.cost = neg_loglikelihood(self.p_y_given_x, self.y)
        self.cost = self.cost

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.pred = aet.argmax(self.p_y_given_x, axis=1)

In [3]:
# train function
model = cmt.train(MNLmodel, db, optimizer=Adam, batch_size=256, lr_init=0.01, max_epoch=4)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

result = Results(model, db, show_weights=True)

Building model...
dataset: swissmetro (6768)
batch size: 256
batches per epoch: 26
validation frequency: 26

Training model...


Loglikelihood:  -5648.636579  Score: 0.612
Epoch    4/4: 100%|██████████| 104/104 [00:03<00:00, 31.8it/s, Patience=2%]

Optimization complete with accuracy of 61.229%
 with maximum loglikelihood reached @ epoch 4.


Results for model myModel
Number of Beta parameters: 4
Total size of Neural Net weights: 60
Sample size: 6768
Init loglikelihood: -7098.896
Final loglikelihood: -5648.637
Likelihood ratio test: 2900.519
Accuracy: 61.229%
Rho square: 0.204
Rho bar square: 0.195
Akaike Information Criterion: 11425.27
Bayesian Information Criterion: 11861.75
Final gradient norm: 0.166

              Value   Std err     t-test   p-value Rob. Std err Rob. t-test Rob. p-value
asc_car    0.170372  0.038788   4.392447  0.000011     0.278241    0.612319     0.540327
asc_sm          0.0         -          -         -            -           -            -
asc_train  -0.41081  0.051326  -8.003997       0.0     0.567376   -0.724052     0.469034
b_cost     0.026847  0.003675   7.304829       0.0     0.011668    2.300857       0.0214
b_time    -0.516164  0.043588 -11.842014       0.0      0.50786   -1.016351     0.309462 

ResNet_01a (3, 10) init: random
[[-0.05209099 -0.81983533 -0.33586673 -0.66219017 -0.7612131  -